# Network analysis - World of Warcraft

## Get data from Battle.net

#### Setup connection

In [1]:
with open('battle_net_api.key', 'r') as f:
    key = f.read()

#### Import API Key

In [2]:
import battlenet_api
reload(battlenet_api)

<module 'battlenet_api' from 'battlenet_api.pyc'>

In [3]:
bn = battlenet_api.BattleNet(public_key=key)

#### Get data for a PVE Realm

In [4]:
realms = bn.realms()

In [5]:
print "Number of Realms: %d" % len(realms)

Number of Realms: 246


In [6]:
realms_pve = [r for r in realms if r['type']=='pve']

In [7]:
print "Number of PVE Realms: %d" % len(realms_pve)

Number of PVE Realms: 117


In [8]:
realm = realms_pve[0]

In [9]:
print "Selected Realm: %s" % realm['name']

Selected Realm: Aerie Peak


#### Get data for Challenges in the selected Realm

In [61]:
challenges = bn.challenges(realm['name'])

In [62]:
print "Number of Challenges: %d" % len(challenges)

Number of Challenges: 17


In [71]:
for m in challenges[0]['groups'][0]['members']:
    if m.has_key('character'):
        print m['character']['name']
    else:
        print '--Name Error--'

Primè
Xjuice
--Name Error--
Shots
Sidebüshlol


In [12]:
groups = challenges[0]['groups']

In [13]:
print "Number of groups per Challenge: %d" % len(groups)

Number of groups per Challenge: 100


#### Figure out distribution of characters in groups

In [14]:
import numpy as np

In [56]:
for challenge in challenges:
    num_members = []  # list of num. of members
    for group in challenge['groups']:
        names = []
        members = group['members']
        for m in members:
            desc.add(m['spec']['icon'])
            if m.has_key('character'):
                 names += [m['character']['name']]
        num_members += [len(names)]
    print "Challenge %s group members: %0.2f average and %0.2f std" % \
    (challenge['map']['name'], np.mean(num_members), np.std(num_members))

Challenge Auchindoun group members: 4.22 average and 0.83 std
Challenge Bloodmaul Slag Mines group members: 4.18 average and 0.83 std
Challenge Gate of the Setting Sun group members: 3.10 average and 1.14 std
Challenge Grimrail Depot group members: 4.18 average and 0.85 std
Challenge Iron Docks group members: 4.36 average and 0.81 std
Challenge Mogu'shan Palace group members: 3.08 average and 1.13 std
Challenge Scarlet Halls group members: 3.14 average and 1.15 std
Challenge Scarlet Monastery group members: 2.90 average and 1.15 std
Challenge Scholomance group members: 2.95 average and 1.13 std
Challenge Shado-Pan Monastery group members: 3.20 average and 1.19 std
Challenge Shadowmoon Burial Grounds group members: 4.35 average and 0.79 std
Challenge Siege of Niuzao Temple group members: 2.87 average and 1.18 std
Challenge Skyreach group members: 4.21 average and 0.84 std
Challenge Stormstout Brewery group members: 3.10 average and 1.09 std
Challenge Temple of the Jade Serpent group mem

In [60]:
challenge.keys()

[u'map', u'realm', u'groups']

In [59]:
group

{u'date': u'2016-01-06T09:29:09.000Z',
 u'faction': u'alliance',
 u'isRecurring': True,
 u'medal': u'GOLD',
 u'members': [{u'spec': {u'backgroundImage': u'bg-deathknight-blood',
    u'description': u'A dark guardian who manipulates and corrupts life energy to sustain herself in the face of an enemy onslaught.',
    u'icon': u'spell_deathknight_bloodpresence',
    u'name': u'Blood',
    u'order': 0,
    u'role': u'TANK'}},
  {u'character': {u'achievementPoints': 9440,
    u'battlegroup': u'Bloodlust',
    u'class': 11,
    u'gender': 1,
    u'lastModified': 0,
    u'level': 100,
    u'name': u'Ctrlv',
    u'race': 4,
    u'realm': u'Tichondrius',
    u'spec': {u'backgroundImage': u'bg-druid-balance',
     u'description': u'Can take on the form of a powerful Moonkin, balancing the power of Arcane and Nature magic to destroy enemies at a distance.',
     u'icon': u'spell_nature_starfall',
     u'name': u'Balance',
     u'order': 0,
     u'role': u'DPS'},
    u'thumbnail': u'tichondrius/19

#### Create a network of characters

In [17]:
import networkx as nx

In [18]:
import algos
reload(algos)

<module 'algos' from 'algos.pyc'>

In [19]:
g = algos.generate_team_graph(challenges)

## Algebraic Connectivity of network and performance

#### Calculate correlation between algebraic connectivity and performance

In [20]:
from scipy.stats import pearsonr

In [21]:
ac, time = algos.get_metrics(challenges, g)

In [22]:
ac = np.array(ac)
time = np.array(time)
r, p = pearsonr(ac, time)

In [23]:
print "Algebraic Connectivity vs. challenge completion time: Pearson coeff = %0.2f, p-value = %0.6f" % (r, p)

Algebraic Connectivity vs. challenge completion time: Pearson coeff = -0.12, p-value = 0.000001


#### Calculate correlation between algebraic connectivity and performance for each challenge

In [25]:
print "Algebraic Connectivity vs. challenge completion time"

for challenge in challenges:
    ac, time = algos.get_metrics(challenges, g)
    ac = np.array(ac)
    time = np.array(time)
    r, p = pearsonr(ac, time)
    print "%s: Pearson coeff = %0.2f, p-value = %0.6f" % (challenge['map']['name'], r, p)

Algebraic Connectivity vs. challenge completion time
Auchindoun: Pearson coeff = -0.12, p-value = 0.000001
Bloodmaul Slag Mines: Pearson coeff = -0.12, p-value = 0.000001
Gate of the Setting Sun: Pearson coeff = -0.12, p-value = 0.000001
Grimrail Depot: Pearson coeff = -0.12, p-value = 0.000001
Iron Docks: Pearson coeff = -0.12, p-value = 0.000001
Mogu'shan Palace: Pearson coeff = -0.12, p-value = 0.000001
Scarlet Halls: Pearson coeff = -0.12, p-value = 0.000001
Scarlet Monastery: Pearson coeff = -0.12, p-value = 0.000001
Scholomance: Pearson coeff = -0.12, p-value = 0.000001
Shado-Pan Monastery: Pearson coeff = -0.12, p-value = 0.000001
Shadowmoon Burial Grounds: Pearson coeff = -0.12, p-value = 0.000001
Siege of Niuzao Temple: Pearson coeff = -0.12, p-value = 0.000001
Skyreach: Pearson coeff = -0.12, p-value = 0.000001
Stormstout Brewery: Pearson coeff = -0.12, p-value = 0.000001
Temple of the Jade Serpent: Pearson coeff = -0.12, p-value = 0.000001
The Everbloom: Pearson coeff = -0.1

#### Calculate correlation between degree and performance

#### Calculate correlation between algebraic connectivity and performance for all PVE Realms

In [24]:
import time

In [118]:
corr = []  # list of tuples [(realm name, corr coeff, p value), ...]
for realm in realms_pve:
    realm_name = realm['name']
#     print realm_name
    try:
        challenges = bn.challenges(realm_name)
        g = algos.generate_team_graph(challenges)
        ac, c_time = algos.get_metrics(challenges, g)
        ac = np.array(ac)
        c_time = np.array(c_time)
        r, p = pearsonr(ac, c_time)
        corr += [(realm_name, r, p)]
        print "Realm: %s, \t Pearson coeff = %0.2f, \t p-value = %0.6f" % (realm_name, r, p)    
    except:
        print "Error with '%s'" % realm_name
    time.sleep(1)
#     break

Realm: Aerie Peak, 	 Pearson coeff = -0.12, 	 p-value = 0.000002
Realm: Aggramar, 	 Pearson coeff = -0.09, 	 p-value = 0.000965
Realm: Alexstrasza, 	 Pearson coeff = -0.05, 	 p-value = 0.046314
Realm: Alleria, 	 Pearson coeff = -0.08, 	 p-value = 0.001852
Realm: Aman'Thul, 	 Pearson coeff = -0.17, 	 p-value = 0.000000
Realm: Antonidas, 	 Pearson coeff = 0.15, 	 p-value = 0.000000
Error with 'Anvilmar'
Error with 'Arathor'
Error with 'Area 52'
Error with 'Arygos'
Error with 'Azjol-Nerub'
Realm: Azuremyst, 	 Pearson coeff = -0.06, 	 p-value = 0.013243
Error with 'Baelgun'
Realm: Blackhand, 	 Pearson coeff = -0.06, 	 p-value = 0.026009
Error with 'Blade's Edge'
Realm: Bladefist, 	 Pearson coeff = -0.06, 	 p-value = 0.028214
Error with 'Bloodhoof'
Realm: Borean Tundra, 	 Pearson coeff = 0.02, 	 p-value = 0.537992
Error with 'Bronzebeard'
Realm: Caelestrasz, 	 Pearson coeff = -0.25, 	 p-value = 0.000000
Realm: Cairne, 	 Pearson coeff = -0.05, 	 p-value = 0.041315
Error with 'Cenarius'
Error

In [126]:
for realm_name, r, p in corr:
    print "Realm: %s,  \t Pearson coeff = %0.2f,  \t p-value = %0.6f" % (realm_name, r, p)    

Realm: Aerie Peak,  	 Pearson coeff = -0.12,  	 p-value = 0.000002
Realm: Aggramar,  	 Pearson coeff = -0.09,  	 p-value = 0.000965
Realm: Alexstrasza,  	 Pearson coeff = -0.05,  	 p-value = 0.046314
Realm: Alleria,  	 Pearson coeff = -0.08,  	 p-value = 0.001852
Realm: Aman'Thul,  	 Pearson coeff = -0.17,  	 p-value = 0.000000
Realm: Antonidas,  	 Pearson coeff = 0.15,  	 p-value = 0.000000
Realm: Azuremyst,  	 Pearson coeff = -0.06,  	 p-value = 0.013243
Realm: Blackhand,  	 Pearson coeff = -0.06,  	 p-value = 0.026009
Realm: Bladefist,  	 Pearson coeff = -0.06,  	 p-value = 0.028214
Realm: Borean Tundra,  	 Pearson coeff = 0.02,  	 p-value = 0.537992
Realm: Caelestrasz,  	 Pearson coeff = -0.25,  	 p-value = 0.000000
Realm: Cairne,  	 Pearson coeff = -0.05,  	 p-value = 0.041315
Realm: Darrowmere,  	 Pearson coeff = -0.08,  	 p-value = 0.002386
Realm: Dawnbringer,  	 Pearson coeff = -0.19,  	 p-value = 0.000000
Realm: Doomhammer,  	 Pearson coeff = -0.03,  	 p-value = 0.170234
Realm

## Character vector and team performance

In [127]:
challenge

{u'groups': [{u'date': u'2015-11-04T07:45:50.000Z',
   u'faction': u'alliance',
   u'isRecurring': False,
   u'medal': u'GOLD',
   u'members': [{u'character': {u'achievementPoints': 7415,
      u'battlegroup': u'Vindication',
      u'class': 6,
      u'gender': 0,
      u'guild': u'Gear Grind',
      u'guildRealm': u'Aerie Peak',
      u'lastModified': 0,
      u'level': 100,
      u'name': u'Prim\xe8',
      u'race': 1,
      u'realm': u'Aerie Peak',
      u'spec': {u'backgroundImage': u'bg-deathknight-blood',
       u'description': u'A dark guardian who manipulates and corrupts life energy to sustain herself in the face of an enemy onslaught.',
       u'icon': u'spell_deathknight_bloodpresence',
       u'name': u'Blood',
       u'order': 0,
       u'role': u'TANK'},
      u'thumbnail': u'ulduar/205/124215757-avatar.jpg'},
     u'spec': {u'backgroundImage': u'bg-deathknight-blood',
      u'description': u'A dark guardian who manipulates and corrupts life energy to sustain herself in t

#### Role based vector

In [139]:
roles = []
for group in challenge['groups']:
    for member in group['members']:
        roles += [member['spec']['role']]

In [140]:
levels = []
for group in challenge['groups']:
    for member in group['members']:
        try:
            levels += [member['character']['level']]
        except:
            pass

In [135]:
print "Types of roles: %s" % ', '.join(set(roles))

Types of roles: DPS, TANK, HEALING


In [141]:
set(levels)

{100}

In [136]:
roles = []
for group in challenge['groups']:
    print ' '.join(m['spec']['role'] for m in group['members'])

TANK HEALING DPS DPS DPS
TANK TANK HEALING DPS DPS
TANK DPS DPS DPS DPS
TANK HEALING DPS DPS DPS
TANK HEALING DPS DPS DPS
TANK DPS DPS DPS DPS
TANK HEALING DPS DPS DPS
TANK DPS DPS DPS DPS
TANK HEALING DPS DPS DPS
TANK DPS DPS DPS DPS
TANK HEALING DPS DPS DPS
TANK TANK TANK DPS DPS
TANK TANK DPS DPS DPS
TANK DPS DPS DPS DPS
TANK HEALING DPS DPS DPS
TANK TANK DPS DPS DPS
TANK HEALING DPS DPS DPS
TANK HEALING DPS DPS DPS
TANK HEALING DPS DPS DPS
TANK HEALING DPS DPS DPS
TANK HEALING DPS DPS DPS
TANK DPS DPS DPS DPS
TANK HEALING DPS DPS DPS
TANK HEALING DPS DPS DPS
TANK DPS DPS DPS DPS
TANK HEALING DPS DPS DPS
TANK TANK DPS DPS DPS
TANK TANK DPS DPS DPS
TANK HEALING DPS DPS DPS
TANK DPS DPS DPS DPS
TANK DPS DPS DPS DPS
TANK DPS DPS DPS DPS
TANK TANK DPS DPS DPS
TANK HEALING DPS DPS DPS
TANK HEALING DPS DPS DPS
TANK HEALING DPS DPS DPS
TANK DPS DPS DPS DPS
TANK DPS DPS DPS DPS
TANK HEALING DPS DPS DPS
TANK HEALING DPS DPS DPS
TANK TANK DPS DPS DPS
TANK HEALING DPS DPS DPS
TANK DPS DPS DPS 